# 과제 1.1

네이버의 주식 사이트에서 제공하는 시가총액 순 데이터는 여기에서 얻을 수 있다. 위 그림은 KOSPI에 상장된 2,000여 개의 주식 중에서 10개 주식에 대한 자료이다.   
* 모든 주식에 대한 자세한 정보를 얻으려면 어떤 데이터를 반드시 수집해야 할까?   
* 수집할 필요가 없는 데이터는 무엇인가?  
웹 스크래핑으로 데이터를 얻어서 아래 CSV 파일 형식으로 저장하는 코드를 작성하라.   또한 작성한 코드를 실행한 결과인 CSV 파일을 제출하라.   

(hint):   
KOSPI 주식 2,161개에 대한 종목명, 액면가, code 정보이다. code는 기본적으로 6자리인데 아래 테이블에는 앞 숫자가 0인 경우 생략되었다 (예: 삼성전자 005930 -> 5930). kospi.csv 파일을 다운받아 보면 이를 확인할 수 있다.    

시가총액 순이므로 날짜에 따라 순서가 달라질 수 있다.

In [ ]:
# 필요한 패키지 불러오기
library(tidyverse)  # 데이터 처리 및 조작을 위한 패키지
library(rvest)      # 웹 스크래핑을 위한 패키지

# 코스피 종목 코드가 있는 네이버 금융 페이지 URL
kospi_url <- "https://finance.naver.com/sise/sise_market_sum.naver?&page="

# 페이지 수를 가져오는 함수 정의
get_total_page <- function(base_url){
  url <- paste(base_url, 1, sep="")  # 첫 번째 페이지의 URL 생성
  html <- read_html(url, encoding="euc-kr")  # HTML 페이지 읽기

  # 페이지 수 추출
  total_page <- html %>%
    html_node(".pgRR") %>%
    html_node("a") %>%
    html_attr('href') %>%
    str_split("page=") %>%
    unlist() %>%
    .[2] %>%
    as.numeric()  # 숫자로 변환하여 반환

  return(total_page)
}

# 페이지별 종목 코드를 가져오는 함수 정의
get_code <- function(base_url, page){
  url <- paste(base_url, page, sep="")  # 현재 페이지의 URL 생성
  html <- read_html(url, encoding="euc-kr")  # HTML 페이지 읽기

  # 종목 코드 추출
  code <- html %>%
    html_nodes("table") %>%
    "["(2) %>%
    html_nodes("td") %>%
    html_nodes("a") %>%
    html_attr("href") %>%
    str_split("=") %>%
    unlist() %>%
    .[c(FALSE, TRUE, FALSE, FALSE)]

  # 종목명과 액면가 데이터 추출 후 데이터프레임 생성
  df <- html %>%
    html_nodes("table") %>%
    .[2] %>%
    html_table() %>%
    data.frame() %>%
    filter(!is.na(N)) %>%
    select(c(종목명, 액면가)) %>%
    mutate(액면가=str_replace_all(액면가, ",", ""))  # 쉼표 제거

  # 종목 코드를 데이터프레임에 추가
  df <- cbind(df, code)

  return(df)
}

# 코스피 총 페이지 수 가져오기
kospi_total_page <- get_total_page(kospi_url)

# 방법 1: lapply를 사용하여 각 페이지의 종목 코드 가져오기
kospi_list <- lapply(1:kospi_total_page, function(page) get_code(kospi_url, page))

# 방법 2: 반복문을 사용하여 각 페이지의 종목 코드 가져오기
kospi_list <- list(kospi_total_page)
for(page in 1:kospi_total_page) kospi_list[[page]] <- data.frame(get_code(kospi_url, page))

# 리스트를 데이터프레임으로 변환
kospi_df <- do.call(rbind, kospi_list)

# CSV 파일로 저장
write_csv(kospi_df, "~/금융통계/kospi_code.csv")

###############################################

# 코스닥 종목 코드가 있는 네이버 금융 페이지 URL
kosdaq_url <- "https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page="

# 코스닥 총 페이지 수 가져오기
kosdaq_total_page <- get_total_page(kosdaq_url)

# 각 페이지의 종목 코드 가져오기
kosdaq_list <- lapply(1:kosdaq_total_page, function(page) get_code(kosdaq_url, page))

# 리스트를 데이터프레임으로 변환
kosdaq_df <- do.call(rbind, kosdaq_list)

# CSV 파일로 저장
write_csv(kosdaq_df, "~/금융통계/kosdaq_code.csv")